In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver

In [2]:
def simple_get(url, load_time=1):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None.
    """
    driver = webdriver.Chrome(executable_path=r"C:\Users\usuario\SportComparator\chromedriver.exe")
    driver.implicitly_wait(5)
    driver.get(url)
    driver.implicitly_wait(load_time)
     
    return driver.page_source

In [3]:
def barrabes_get_page_num(raw_html):
    
    html = BeautifulSoup(raw_html, 'lxml')
    
    pag = html.find("div", {"id": "txtPaginacion"})
    
    return pag.find_all('b')[2].text

In [28]:
def barrabes_extract_info(raw_html, category, family, section):
    
    html = BeautifulSoup(raw_html, 'lxml')
    names = []
    brand = []
    url = []
    price = []
    old_price = []
    sub = []
    price_tmp = 0
    
    for box in html.findAll('div', {'class': 'boxProducto'}):
        
        product = box.find('div', {'class': 'boxNombreProd'})
        
        for a in product.findAll('a'):
            names.append(a.find('b').text)
            brand.append(a.text.replace(a.find('b').text,''))
            url.append(a['href'])
        
        for span in box.findAll('span', {'class': 'texto12'}):
            price.append(span.text)
            price_tmp = span.text
        
        if box.findAll('span', {'class': 'precioOferta'}):
            for span in box.findAll('span', {'class': 'precioAnterior'}):
                old_price.append(span.text)
        else:
            old_price.append(price_tmp)
            
    for box in html.findAll('div', {'class': 'boxProductof'}):
        
        product = box.find('div', {'class': 'boxNombreProd'})
        
        for a in product.findAll('a'):
            names.append(a.find('b').text)
            brand.append(a.text.replace(a.find('b').text,''))
            url.append(a['href'])
        
        for span in box.findAll('span', {'class': 'texto12'}):
            price.append(span.text)
            price_tmp = span.text
        
        if box.findAll('span', {'class': 'precioOferta'}):
            for span in box.findAll('span', {'class': 'precioAnterior'}):
                old_price.append(span.text)
        else:
            old_price.append(price_tmp)
    
    df = pd.DataFrame(list(zip(names, brand, price, old_price, url)))
    df.columns = ['Name', 'Brand', 'Price', 'Old Price', 'Url']
    
    df['Category'] = category
    df['Family'] = family
    df['Section'] = section
    
    return df

In [27]:
def barrabes_get_family_items(dictionary, category, section):  
    
    df = pd.DataFrame()
    
    for family,url in dictionary.items():

        raw_html = simple_get(url, 7)
        num_pag = barrabes_get_page_num(raw_html)
        
        df_tmp = pd.DataFrame()

        for i in range(int(num_pag)):   
            raw_html = simple_get(url + str(i+1), 7)
            df_tmp = barrabes_extract_info(raw_html, category, family, section)
            df = df.append(df_tmp)
        
    return df
        

In [29]:
from dictionaries_men import chaquetas, chalecos, pantalones, camisetas, ropa_interior, guantes_y_manoplas
from dictionaries_men import gorros_y_tubulares, barrancos, calcetines, calzado, lifestyle

df = pd.DataFrame()

df_tmp = barrabes_get_family_items(chaquetas, "Caquetas", "Hombre")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(chalecos, "Chalecos", "Hombre")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(pantalones, "Pantalones", "Hombre")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(camisetas, "Camisetas", "Hombre")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(ropa_interior, "Ropa interior", "Hombre")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(guantes_y_manoplas, "Guantes y manoplas", "Hombre")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(gorros_y_tubulares, "Gorros y tubulares", "Hombre")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(barrancos, "Barrancos", "Hombre")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(calcetines, "Calcetines", "Hombre")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(calzado, "Calzado", "Hombre")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(lifestyle, "Lifestyle", "Hombre")
df = df.append(df_tmp)

df = df.reset_index(drop=True)



KeyboardInterrupt: 

In [22]:
from dictionaries_women import chaquetas, chalecos, pantalones, camisetas, ropa_interior, guantes_y_manoplas
from dictionaries_women import gorros_y_tubulares, barrancos, calcetines, calzado, lifestyle

df_tmp = barrabes_get_family_items(chaquetas, "Caquetas", "Mujer")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(chalecos, "Chalecos", "Mujer")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(pantalones, "Pantalones", "Mujer")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(camisetas, "Camisetas", "Mujer")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(ropa_interior, "Ropa interior", "Mujer")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(guantes_y_manoplas, "Guantes y manoplas", "Mujer")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(gorros_y_tubulares, "Gorros y tubulares", "Mujer")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(barrancos, "Barrancos", "Mujer")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(calcetines, "Calcetines", "Mujer")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(calzado, "Calzado", "Mujer")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(lifestyle, "Lifestyle", "Mujer")
df = df.append(df_tmp)

df = df.reset_index(drop=True)


In [ ]:
from dictionaries_unisex import relojes, gafas, accesorios_y_limpieza, capas_y_ponchos, monos

df_tmp = barrabes_get_family_items(relojes, "Relojes", "Unisex")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(gafas, "Gafas", "Unisex")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(accesorios_y_limpieza, "Accesorios y limpieza", "Unisex")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(capas_y_ponchos, "Capas y ponchos", "Unisex")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(monos, "Monos", "Unisex")
df = df.append(df_tmp)

df = df.reset_index(drop=True)

In [ ]:
from dictionaries_esqui import esqui_de_montaña_y_freeride, esqui_alpino_y_freestyle, esqui_de_fondo, snowboard
from dictionaries_esqui import raquetas, cascos_y_protecciones, seguridad, accesorios_y_mantenimiento

df_tmp = barrabes_get_family_items(esqui_de_montaña_y_freeride, "Esqui de montaña y freeride", "Esqui")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(esqui_alpino_y_freestyle, "Esqui alpino y freestyle", "Esqui")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(esqui_de_fondo, "Esqui de fondo", "Esqui")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(snowboard, "Snowboard", "Esqui")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(raquetas, "Raquetas", "Esqui")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(cascos_y_protecciones, "Cascos y protecciones", "Esqui")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(seguridad, "Seguridad", "Esqui")
df = df.append(df_tmp)

df_tmp = barrabes_get_family_items(accesorios_y_mantenimiento, "Accesorios y mantenimiento", "Esqui")
df = df.append(df_tmp)

df = df.reset_index(drop=True)

In [30]:
print(df.shape)

(0, 0)


In [26]:
df.to_csv('BarrabesProducts.csv', sep='\t', encoding='utf-8')

,Name,Brand,Price,Old Price,Url,Category,Family,Section
0,Squasmish Hoody,Arc'teryx,120.00 €,150.00 €,/arcteryx-squasmish-hoody/p-80179?idvariedad=3...,Caquetas,Aeróbicas,Hombre
1,Pierra Ment Jkt,Millet,107.90 €,179.90 €,/millet-pierra-ment-jkt/p-72897?idvariedad=295...,Caquetas,Aeróbicas,Hombre
2,Fast Wing Jacket,Salomon,54.49 €,100.00 €,/salomon-fast-wing-jacket/p-81711?idvariedad=3...,Caquetas,Aeróbicas,Hombre
3,Fast Wing Aero,Salomon,78.00 €,130.00 €,/salomon-fast-wing-aero/p-72999?idvariedad=295...,Caquetas,Aeróbicas,Hombre
4,Drew Peak WindWall Jacket,The North Face,71.80 €,90.00 €,/the-north-face-drew-peak-windwall-jacket/p-95...,Caquetas,Aeróbicas,Hombre
5,Keiro Diad WD Jacket,The North Face,60.00 €,100.00 €,/the-north-face-keiro-diad-wd-jacket/p-93930?i...,Caquetas,Aeróbicas,Hombre
6,Squamish Hoody,Arc'teryx,128.00 €,160.00 €,/arcteryx-squamish-hoody/p-46654?idvariedad=18...,Caquetas,Aeróbicas,Hombre
7,Nodin Jacket,Arc'teryx,80.00 €,160.00 €,/arcteryx-nodin-jacket/p-78089?idvariedad=3106...,Caquetas,Aeróbicas,Hombre
8,UltraLight Windproof,Raidlight,47.40 €,79.00 €,/raidlight-ultralight-windproof/p-82685?idvari...,Caquetas,Aeróbicas,Hombre
9,PDG U Jacket,Dynafit,101.25 €,180.00 €,/dynafit-pdg-u-jacket/p-49839?idvariedad=25942...,Caquetas,Aeróbicas,Hombre
